In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PATH = '/content/drive/My Drive/'

list_ = ['ramen', 'star']
num = len(list_)

In [0]:
import numpy as np
import keras
import keras.utils
from keras import utils as np_utils

X_train, X_test, y_train, y_test = np.load(PATH + 'image_files.npy', allow_pickle=True)
#print(X_train[1])
#########one-hot化すりゅ################
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

########正規化#######################
X_train = X_train.astype('float') / 255.0
X_test = X_test.astype('float') / 255.0

In [0]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD

In [0]:
IMAGE_SIZE = 224

In [0]:
vgg16_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)   
)





58892288/58889256 [==============================] - 1s 0us/step








In [0]:
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(
    inputs=vgg16_model.input,
    outputs=top_model(vgg16_model.output)
)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
for layer in model.layers[:15]:
    layer.trainable = False

In [0]:
opt = SGD(lr=1e-4, momentum=0.9)
#####2クラス分類に設定#########
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=6) 

Epoch 1/6
222/222 [==============================] - 1s 5ms/step - loss: 0.0046 - acc: 1.0000
Epoch 2/6
222/222 [==============================] - 1s 3ms/step - loss: 0.0027 - acc: 1.0000
Epoch 3/6
222/222 [==============================] - 1s 3ms/step - loss: 0.0016 - acc: 1.0000
Epoch 4/6
222/222 [==============================] - 1s 3ms/step - loss: 0.0050 - acc: 1.0000
Epoch 5/6
222/222 [==============================] - 1s 3ms/step - loss: 0.0035 - acc: 1.0000
Epoch 6/6
222/222 [==============================] - 1s 3ms/step - loss: 0.0035 - acc: 1.0000


In [0]:
score = model.evaluate(X_test, y_test, batch_size=32)
print('loss: {0} - acc: {1}'.format(score[0], score[1]))

74/74 [==============================] - 1s 9ms/step
loss: 0.07835521849149184 - acc: 0.9864864864864865


In [0]:
model.save(PATH + 'vgg16_transfer.h5')